In [1]:
#!/usr/declass/env/bin/python
# -*- coding: utf-8 -*-

import os
import sys
import pymysql
import codecs
import ConfigParser
import pandas as pd
from pandas.api.types import is_string_dtype


config = ConfigParser.RawConfigParser()
config.readfp(open(r'../mylogin.cnf'))
db_user = config.get('client', 'user')
db_pass = config.get('client', 'password')

cables_db = pymysql.connect(host='history-lab.org', user=db_user, passwd=db_pass, db="declassification_cables", charset='utf8')

In [2]:
tables = pd.read_sql("SHOW TABLES;", con=cables_db)
tables = list(tables['Tables_in_declassification_cables'].values.astype(str))
tables

['classification_countries',
 'classification_doc',
 'classifications',
 'concept_doc',
 'concepts',
 'countries',
 'country_doc',
 'doc_counts',
 'docs',
 'from_to_sum',
 'network_docs',
 'network_nodes',
 'office_doc',
 'offices',
 'person_doc',
 'persons',
 'reference_doc',
 'tag_doc',
 'tag_doc_staging',
 'tagname_doc',
 'tagnames',
 'tags',
 'tags_staging',
 'tokens',
 'top_classifications',
 'top_countries',
 'top_network',
 'top_persons',
 'top_topics',
 'topic_doc',
 'topic_token',
 'topics',
 'urgency',
 'urgency_doc']

In [3]:
def markdown_table(df):
    fmt = ['---' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    return df_formatted.to_csv(sep="|", index=False, encoding='utf-8', quotechar="*")

In [4]:
def preview(tables):
    all_tables_string = "### `declassification_cables` preview \n\n"
    for table in tables:
        exec("df_head = pd.read_sql('SELECT * FROM ' + table + ' LIMIT 5;', con=cables_db)")
        for col in df_head.columns:
            if is_string_dtype(df_head[col]):
                df_head[col] = df_head[col].str[:200].replace('\n',' ', regex=True)
        all_tables_string += "#### `" + table + "`\n\n" + markdown_table(df_head) + "\n\n"
    return all_tables_string

In [5]:
def print_to_md(preview):
    result_string = "# Cables Collection Overview — " + "\n\n\n" + preview
    results_filepath = "cables-preview.md"
    print "Writing results of ingestion to " + results_filepath + "..."

    try:
        os.remove(results_filepath)
    except OSError:
        pass

    f = codecs.open(results_filepath, encoding='utf-8', mode="w+")
    f.write(unicode(result_string,'utf-8'))

In [6]:
# print_to_md(preview(tables))

In [7]:
countries = pd.read_sql("SELECT * FROM countries;", con=cables_db)
countries.head()

,id,name,deleted,official,tag_id
0,004,Afghanistan,0.0,1.0,342.0
1,008,Albania,0.0,1.0,343.0
2,010,Antarctica,0.0,1.0,1346.0
3,012,Algeria,0.0,1.0,344.0
4,016,American Samoa,0.0,1.0,345.0


In [8]:
countries[countries['tag_id'].isnull()].head()

,id,name,deleted,official,tag_id
8,031,Azerbaijan,0.0,1.0,NaN
25,080,British Antarctic Territory,1.0,0.0,NaN
32,104,Myanmar,0.0,1.0,NaN
34,112,Belarus,0.0,1.0,NaN
39,132,Cabo Verde,0.0,1.0,NaN


In [9]:
tags = pd.read_sql("SELECT * FROM tags;", con=cables_db)
tags

,id,tag,title,description,class,category,action
0,13,AART,Art In Embassies Program,Use for communications relating to the art-in-...,administration,subject,A/ART
1,14,ABLD,Buildings,Use for (1) matters pertaining to FBO appropri...,administration,subject,A/FBO
2,15,ABUD,Budget Services and Financial Systems,Use for matters relating to budget operations ...,administration,subject,A/BF
3,16,ACLM,Claims Against the USG,"Use for all claims, suits, etc brought against...",administration,subject,None
4,17,ACMM,Committes,Use for matters pertaining to the establishmen...,administration,subject,M/MO
5,18,ACMS,COMSEC Material,Use for communications discussing (a) shipment...,administration,subject,None
6,19,ACOU,Courier Operations,Use for all matters pertaining to courier oper...,administration,subject,None
7,20,ADTO,Domestic Telecommunications Operations,Use for communications discussing all aspects ...,administration,subject,None
8,21,AEMR,Emergency and Evacuation,"Use for all emergency and evacutation matters,...",administration,subject,OPR/WLG
9,22,AFAC,CRU/CCC Facilities (Communications and Records...,Use for communications discussing (a) the phys...,administration,subject,None


In [10]:
tag_doc = pd.read_sql("SELECT * FROM tag_doc LIMIT 100;", con=cables_db)

In [11]:
tag_doc

,tag_id,doc_id
0,13,1973KHARTO02100
1,13,1973LENING00586
2,13,1973MOGADI01397
3,13,1973NASSAU01116
4,13,1973NICOSI02503
5,13,1973PORTO01011
6,13,1973STATE155679
7,13,1973STATE186522
8,13,1973STATE209366
9,13,1973TEHRAN06634
